In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from collections import Counter
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D
from keras.layers.core import Dense, Dropout
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

C:\Users\lenovo\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\lenovo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\lenovo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\lenovo\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a futur

In [15]:
path = r'E:/Data Sets'
df = pd.read_csv(os.path.join(path, 'Dataset-weather-prediction-dataset-processed.csv'))
df1 = pd.read_csv(os.path.join(path, 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))
df.drop(columns=['DATE', 'MONTH'],inplace=True)
df1.drop(columns=['DATE'],inplace=True)
list3 = []
for i in df.columns.to_list():
    if (i.split('_', 1)[0].replace('.', '').upper() == 'ROMA' or i.split('_', 1)[0].replace('.', '').upper() == 'TOURS' or i.split('_', 1)[0].replace('.', '').upper() == 'GDANSK'):
        list3.append(i)

df.drop(columns = list3, inplace = True)
df.head()

,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,2.1,0.85,1.018,0.32,0.09,0,0.7,6.5,0.8,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,6.1,3.3,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,8,2.1,0.90,1.018,0.18,0.30,0,0.0,8.5,5.1,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,3,2.1,0.92,1.018,0.58,0.00,0,4.1,6.3,3.8,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,6,2.1,0.95,1.018,0.65,0.14,0,5.4,3.0,-0.7,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4


In [16]:
df.isnull().sum().sort_values(ascending=False)

VALENTIA_temp_max       0
DEBILT_temp_mean        0
HEATHROW_pressure       0
HEATHROW_humidity       0
HEATHROW_cloud_cover    0
                       ..
MADRID_temp_min         0
MADRID_temp_mean        0
MADRID_sunshine         0
MADRID_precipitation    0
BASEL_cloud_cover       0
Length: 147, dtype: int64

In [17]:
list1 = [text.split('_',1)[1] for text in df.columns.to_list()]
Counter(list1)

Counter({'cloud_cover': 14,
         'wind_speed': 9,
         'humidity': 14,
         'pressure': 14,
         'global_radiation': 15,
         'precipitation': 15,
         'snow_depth': 6,
         'sunshine': 15,
         'temp_mean': 15,
         'temp_min': 15,
         'temp_max': 15})

In [18]:
df = df[df.columns.drop(list(df.filter(regex = 'wind_speed')))]
df = df.loc[:,~df.columns.str.endswith('snow_depth')]
df.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4


In [19]:
l2 = ['cloud_cover','humidity','pressure']
list3 = []
for i in df.columns.to_list():
    list3.append(i.split('_', 1)[0].replace('.', '').upper())
l3 = list(set(list3))
l4 = []
for i in l3:
    l4.append(i + '_' + 'cloud_cover')
    l4.append(i + '_' + 'humidity')
    l4.append(i + '_' + 'pressure')
print(l4)

['VALENTIA_cloud_cover', 'VALENTIA_humidity', 'VALENTIA_pressure', 'HEATHROW_cloud_cover', 'HEATHROW_humidity', 'HEATHROW_pressure', 'MUNCHENB_cloud_cover', 'MUNCHENB_humidity', 'MUNCHENB_pressure', 'MADRID_cloud_cover', 'MADRID_humidity', 'MADRID_pressure', 'OSLO_cloud_cover', 'OSLO_humidity', 'OSLO_pressure', 'DUSSELDORF_cloud_cover', 'DUSSELDORF_humidity', 'DUSSELDORF_pressure', 'STOCKHOLM_cloud_cover', 'STOCKHOLM_humidity', 'STOCKHOLM_pressure', 'BELGRADE_cloud_cover', 'BELGRADE_humidity', 'BELGRADE_pressure', 'DEBILT_cloud_cover', 'DEBILT_humidity', 'DEBILT_pressure', 'KASSEL_cloud_cover', 'KASSEL_humidity', 'KASSEL_pressure', 'LJUBLJANA_cloud_cover', 'LJUBLJANA_humidity', 'LJUBLJANA_pressure', 'BASEL_cloud_cover', 'BASEL_humidity', 'BASEL_pressure', 'BUDAPEST_cloud_cover', 'BUDAPEST_humidity', 'BUDAPEST_pressure', 'SONNBLICK_cloud_cover', 'SONNBLICK_humidity', 'SONNBLICK_pressure', 'MAASTRICHT_cloud_cover', 'MAASTRICHT_humidity', 'MAASTRICHT_pressure']


In [20]:
list(set(l4) - set(df.columns.to_list()))

['MUNCHENB_pressure', 'STOCKHOLM_humidity', 'KASSEL_cloud_cover']

In [21]:
df['MUNCHENB_pressure'] = df['SONNBLICK_pressure'] 
df['KASSEL_cloud_cover'] = df['LJUBLJANA_cloud_cover'] 
df['STOCKHOLM_humidity'] = df['OSLO_humidity'] 

In [22]:
print(df.shape)
df.to_csv(os.path.join(path, 'Task-2.2-revised-data.csv'))
df = np.array(df)
df = df.reshape(-1,15,9)

(22950, 135)


In [23]:
df1.shape

(22950, 15)

# Data Splitting

In [12]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(df,df1,random_state = 42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212, 15)
(5738, 15, 9) (5738, 15)


In [13]:
stations = {0: 'BASEL', 1: 'BELGRADE', 2: 'BUDAPEST', 3: 'DEBILT', 4: 'DUSSELDORF', 5: 'HEATHROW', 6: 'KASSEL', 7: 'LJUBLJANA', 8: 'MAASTRICHT', 9: 'MADRID', 10: 'MUNCHENB', 11: 'OSLO', 12: 'SONNBLICK', 13: 'STOCKHOLM', 14: 'VALENTIA'}

def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([stations[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([stations[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

# First layered model

In [76]:
epochs = 30
batch_size = 16
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) 

Instructions for updating:
Colocations handled automatically by placer.


In [77]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 14, 32)            608       
_________________________________________________________________
dense_1 (Dense)              (None, 14, 16)            528       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 7, 16)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 112)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 15)                1695      
Total params: 2,831
Trainable params: 2,831
Non-trainable params: 0
_________________________________________________________________


In [78]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
 - 2s - loss: 8.3475 - accuracy: 0.1328
Epoch 2/30
 - 2s - loss: 7.7354 - accuracy: 0.1584
Epoch 3/30
 - 2s - loss: 7.6002 - accuracy: 0.1737
Epoch 4/30
 - 2s - loss: 7.5247 - accuracy: 0.1756
Epoch 5/30
 - 2s - loss: 7.4784 - accuracy: 0.1752
Epoch 6/30
 - 2s - loss: 7.4409 - accuracy: 0.1753
Epoch 7/30
 - 1s - loss: 7.4140 - accuracy: 0.1762
Epoch 8/30
 - 1s - loss: 7.3939 - accuracy: 0.1757
Epoch 9/30
 - 2s - loss: 7.3683 - accuracy: 0.1804
Epoch 10/30
 - 2s - loss: 7.3536 - accuracy: 0.1813
Epoch 11/30
 - 1s - loss: 7.3331 - accuracy: 0.1850
Epoch 12/30
 - 2s - loss: 7.3239 - accuracy: 0.1861
Epoch 13/30
 - 2s - loss: 7.3050 - accuracy: 0.1924
Epoch 14/30
 - 1s - loss: 7.2860 - accuracy: 0.1959
Epoch 15/30
 - 1s - loss: 7.2738 - accuracy: 0.1981
Epoch 16/30
 - 2s - loss: 7.2582 - accuracy: 0.2020
Epoch 17/30
 - 2s - loss: 7.2454 - accuracy: 0.2060
Epoch 18/30
 - 1s - loss: 7.2226 - accuracy: 0.2115
Epoch 19/30
 - 2s - loss:

In [91]:
# Evaluate
y_test = np.array(y_test)
y_test = y_test.reshape(-1,15)
print(confusion_matrix(y_test, model.predict(X_test)))

Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         259       625       159      89          91       126      15   
BELGRADE        1       415       107      17          15        26       4   
BUDAPEST        1        21        24      12           4         9       0   
DEBILT          1         0         2       7           6         5       4   
DUSSELDORF      0         0         0       0           2         5       0   
HEATHROW        0         2         0       0           0        22       0   
KASSEL          1         1         0       0           1         0       0   
LJUBLJANA       0         3         2       0           0         1       0   
MAASTRICHT      3         0         0       0           1         1       0   
MADRID          0         5         1       0           0         0       0   
MUNCHENB        0         1         0       0       

# Second model (increasing the number of hidden layers to 64)

In [21]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh')) # Options: sigmoid, tanh, softmax, relu

In [22]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 14, 64)            1216      
_________________________________________________________________
dense_3 (Dense)              (None, 14, 16)            1040      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 7, 16)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 112)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 15)                1695      
Total params: 3,951
Trainable params: 3,951
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/30
 - 2s - loss: 24.7089 - accuracy: 0.1879
Epoch 2/30
 - 2s - loss: 28.1655 - accuracy: 0.2038
Epoch 3/30
 - 2s - loss: 21.1572 - accuracy: 0.1986
Epoch 4/30
 - 2s - loss: 30.7632 - accuracy: 0.2564
Epoch 5/30
 - 2s - loss: 30.7688 - accuracy: 0.2785
Epoch 6/30
 - 2s - loss: 30.7688 - accuracy: 0.2800
Epoch 7/30
 - 2s - loss: 30.7688 - accuracy: 0.2831
Epoch 8/30
 - 2s - loss: 30.7688 - accuracy: 0.2844
Epoch 9/30
 - 2s - loss: 30.7688 - accuracy: 0.2866
Epoch 10/30
 - 2s - loss: 30.7688 - accuracy: 0.2889
Epoch 11/30
 - 2s - loss: 30.7669 - accuracy: 0.2903
Epoch 12/30
 - 2s - loss: 30.7669 - accuracy: 0.2900
Epoch 13/30
 - 2s - loss: 30.7669 - accuracy: 0.2914
Epoch 14/30
 - 2s - loss: 30.7669 - accuracy: 0.2917
Epoch 15/30
 - 2s - loss: 30.7669 - accuracy: 0.2919
Epoch 16/30
 - 2s - loss: 30.7660 - accuracy: 0.2964
Epoch 17/30
 - 2s - loss: 30.7660 - accuracy: 0.2949
Epoch 18/30
 - 2s - loss: 30.7660 - accuracy: 0.2948
Epoch 19/30
 - 2s - loss: 30.7688 - accuracy: 0.2947
Ep

In [25]:
y_test = np.array(y_test)
y_test = y_test.reshape(-1,15)
print(confusion_matrix(y_test, model.predict(X_test)))

Pred        BASEL  BUDAPEST  HEATHROW  KASSEL  LJUBLJANA  OSLO  VALENTIA
True                                                                    
BASEL        3651         9         4       1          7     4         6
BELGRADE     1092         0         0       0          0     0         0
BUDAPEST      214         0         0       0          0     0         0
DEBILT         82         0         0       0          0     0         0
DUSSELDORF     29         0         0       0          0     0         0
HEATHROW       82         0         0       0          0     0         0
KASSEL         11         0         0       0          0     0         0
LJUBLJANA      61         0         0       0          0     0         0
MAASTRICHT      9         0         0       0          0     0         0
MADRID        457         0         0       0          0     1         0
MUNCHENB        8         0         0       0          0     0         0
OSLO            5         0         0       0      

In [28]:
model.predict(X_test).shape

(5738, 15)

In [27]:
confusion_matrix(y_test, model.predict(X_test))

Pred,BASEL,BUDAPEST,HEATHROW,KASSEL,LJUBLJANA,OSLO,VALENTIA
True,,,,,,,
BASEL,3651,9,4,1,7,4,6
BELGRADE,1092,0,0,0,0,0,0
BUDAPEST,214,0,0,0,0,0,0
DEBILT,82,0,0,0,0,0,0
DUSSELDORF,29,0,0,0,0,0,0
HEATHROW,82,0,0,0,0,0,0
KASSEL,11,0,0,0,0,0,0
LJUBLJANA,61,0,0,0,0,0,0
MAASTRICHT,9,0,0,0,0,0,0


In [29]:
epochs = 50
batch_size = 32
n_hidden = 128

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) 

In [30]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/50
 - 1s - loss: 8.1103 - accuracy: 0.1715
Epoch 2/50
 - 1s - loss: 7.6965 - accuracy: 0.1692
Epoch 3/50
 - 1s - loss: 7.5903 - accuracy: 0.1740
Epoch 4/50
 - 1s - loss: 7.5036 - accuracy: 0.1823
Epoch 5/50
 - 1s - loss: 7.4488 - accuracy: 0.1882
Epoch 6/50
 - 1s - loss: 7.4000 - accuracy: 0.1877
Epoch 7/50
 - 1s - loss: 7.3522 - accuracy: 0.1938
Epoch 8/50
 - 1s - loss: 7.3054 - accuracy: 0.1921
Epoch 9/50
 - 1s - loss: 7.2636 - accuracy: 0.1870
Epoch 10/50
 - 1s - loss: 7.2299 - accuracy: 0.1910
Epoch 11/50
 - 1s - loss: 7.1924 - accuracy: 0.1947
Epoch 12/50
 - 1s - loss: 7.1628 - accuracy: 0.1970
Epoch 13/50
 - 1s - loss: 7.1306 - accuracy: 0.1986
Epoch 14/50
 - 1s - loss: 7.1092 - accuracy: 0.1976
Epoch 15/50
 - 1s - loss: 7.0812 - accuracy: 0.2069
Epoch 16/50
 - 1s - loss: 7.0497 - accuracy: 0.2020
Epoch 17/50
 - 1s - loss: 7.0259 - accuracy: 0.2098
Epoch 18/50
 - 1s - loss: 7.0041 - accuracy: 0.2120
Epoch 19/50
 - 1s - loss: 6.9823 - accuracy: 0.2155
Epoch 20/50
 - 1s - l

In [31]:
confusion_matrix(y_test, model.predict(X_test))

Pred,BASEL,BELGRADE,BUDAPEST,DEBILT,DUSSELDORF,HEATHROW,KASSEL,LJUBLJANA,MAASTRICHT,MADRID,MUNCHENB,OSLO,STOCKHOLM,VALENTIA
True,,,,,,,,,,,,,,
BASEL,344,928,111,169,110,154,188,217,237,636,133,35,3,417
BELGRADE,1,457,138,40,43,26,19,81,9,215,23,33,3,4
BUDAPEST,1,5,72,3,11,7,4,17,1,77,4,8,3,1
DEBILT,0,0,0,14,16,10,6,0,5,19,0,5,1,6
DUSSELDORF,1,0,0,0,5,2,1,1,1,16,0,1,1,0
HEATHROW,0,2,0,0,0,37,0,1,0,39,0,2,1,0
KASSEL,0,0,0,0,0,0,2,1,0,6,2,0,0,0
LJUBLJANA,1,0,1,0,0,0,0,17,0,40,0,2,0,0
MAASTRICHT,0,0,0,0,1,1,0,0,5,2,0,0,0,0
